# Final code at the last code block

The last two **code blocks** calculates time needed for 2 different approaches to translation
Chinese is the assumed language to be translated to english.

#### Note :-
1. Running `translate` in kaggle or google collab seems more feasible than on local machine since it takes quite a lot of time on local machine with slow hardware.

2. Though its NLP task hence CPU intensive and free tier does not provide with good CPUs. **2 cores 2.30 ghz each in collab and 4 cores 2.30 ghz each in kaggle**.   

3. `googletrans` relies on data fetch from server and does not stress local resources.


[My Collab link](https://colab.research.google.com/drive/1sFpVnbat8oenrNzDLFkUN6j5qt1MQxvU?usp=sharing)

## All Imports

Install the following via pip :-
```bash
pip install xlrd xlwt translate googletrans==4.0.0-rc1 thread
```

In [2]:
!pip install xlrd xlwt translate googletrans==4.0.0-rc1 thread
import time
import os
import thread
import xlrd  # Probably Fastest for reading traditional XLS files since it only does that(no writing)
import xlwt
from translate import Translator as t_trans # local library for offline translation on system
from googletrans import Translator as google_trans
import threading
import multiprocessing

## Loading the dataset

We assume dataset is present in data folder of working directory

In [4]:
# We first read the given XLS file using xlrd
workbook = xlrd.open_workbook("./data/Order Export.xls")
print("The number of worksheets given : {0}".format(workbook.nsheets))
print("Name of Worksheet(s) : {0}".format(workbook.sheet_names()))

# loading the sheet
sheet = workbook.sheet_by_index(0)

print("Total number of rows : ",sheet.nrows,"\nTotal number of columns : ", sheet.ncols)

The number of worksheets given : 1
Name of Worksheet(s) : ['1703575094065_1550538120']
Total number of rows :  109 
Total number of columns :  49


## Viewing a particular row and column
This is just for testing purpose only
1. View data of an entire row
2. View data of an entire column
3. View data of a particular desired cell

#### NOTE :-

Think of it as a simple matrix of data with 0-based indexing for both row and col and access accordingly.

Read more here :- [python-excel.pdf](https://github.com/python-excel/tutorial/raw/master/python-excel.pdf)

In [4]:
print(type(sheet.row(0)), type(sheet.cell(2,1)), type(sheet.col_values(1)), type(sheet.cell_value(3,2)))

print("\n\nData in row 0 i.e. headers : ",sheet.row(0)) # key value pairs
print("\nData in row 1 : ",sheet.row_values(1)) # only values as list

print("\n\nData in column 0 : ",sheet.col(0))
print("\nData in column 1 : ",sheet.col_values(1))

print("\n\nCell value at (1,0) : ", sheet.cell(1,0))
print("\nCell value at (2,1) : ", sheet.cell_value(2,1))


<class 'list'> <class 'xlrd.sheet.Cell'> <class 'list'> <class 'str'>


Data in row 0 i.e. headers :  [text:'订单编号', text:'买家公司名', text:'买家会员名', text:'卖家公司名', text:'卖家会员名', text:'货品总价(元)', text:'运费(元)', text:'涨价或折扣(元)', text:'实付款(元)', text:'订单状态', text:'订单创建时间', text:'订单付款时间', text:'发货方', text:'收货人姓名', text:'收货地址', text:'邮编', text:'联系电话', text:'联系手机', text:'货品标题', text:'单价(元)', text:'数量', text:'单位', text:'货号', text:'型号', text:'Offer ID', text:'SKU ID', text:'物料编号', text:'单品货号', text:'货品种类', text:'买家留言', text:'物流公司', text:'运单号', text:'发票：购货单位名称', text:'发票：纳税人识别号', text:'发票：地址、电话', text:'发票：开户行及账号', text:'发票收取地址', text:'关联编号', text:'代理商姓名', text:'代理商联系方式', text:'是否代发订单', text:'代发服务商id', text:'微商订单号', text:'下单批次号', text:'下游渠道', text:'下游订单号', text:'下单公司主体', text:'发起人登录名', text:'是否发起免密支付(1:淘货源诚e赊免密支付2:批量下单免密支付)']

Data in row 1 :  ['3704877109186608177', '', 'tb582836650', '台州市路桥乐缇电子商务商行', '乐缇贸易', 75.0, 10.0, 0.0, 85.0, '等待买家付款', 45285.7677662037, '', '商家发货', 'BGL海运仓by sea', '广东省 广州市 白云区 太和镇

## Create a output folder in current location if not exists

In [5]:

def create_output_folder():
    folder_name = "output"
    current_location = os.getcwd()
    output_folder_path = os.path.join(current_location, folder_name)

    # Check if the folder exists
    if not os.path.exists(output_folder_path):
        # Create the folder if it doesn't exist
        os.makedirs(output_folder_path)
        print(f"Folder '{folder_name}' created at: {output_folder_path}")
    else:
        print(f"Folder '{folder_name}' already exists at: {output_folder_path}")

# Call the function to create the "output" folder
create_output_folder()

Folder 'output' created at: /content/output


## Converting all chinese data into english and storing in a xls file again

We have few options

1. Project `translate` from PyPi, we use the offline version only since DeepL has limited API callsfree tier

2. Project `googletrans` from PyPi which offers google translate API with unlimited calls.

### Using `translate` module

auto detection has some issues see [stackoverflow](https://stackoverflow.com/questions/71631442/translate-python-not-auto-detecting-language-properly) hence hardcode them

#### without threading

In [6]:
cur_lang = 'zh'
trans_lang = 'en'

translator = t_trans(from_lang = cur_lang, to_lang =  trans_lang)
# print(translator.translate("订单编号"))

start_time = time.time()

data_english_version = []
for row_ind in range(sheet.nrows):
    chinese_data_in_row = sheet.row_values(row_ind)
    english_data_in_row = [translator.translate(str(chinese_text)) for chinese_text in chinese_data_in_row]
    data_english_version.append(english_data_in_row)
    # break

# Writing to output folder
workbook = xlwt.Workbook()
worksheet = workbook.add_sheet("Translated Sheet 1")
for row_index, row_data in enumerate(data_english_version):
    for col_index, cell_value in enumerate(row_data):
        worksheet.write(row_index, col_index, cell_value)
workbook.save("./output/Translated_Exports_via_translate.xls")

end_time = time.time()
trans_time = (end_time - start_time)
print("Total time taken by translate and saving xls file : ", trans_time," seconds")

Total time taken by translate and saving xls file :  2081.246784210205  seconds


#### with threading

tried implementing threading but little speed improved.

In [5]:
def translate_chunk(start_row, end_row):
    # Translates a chunk of rows in a separate thread.
    english_data_chunk = []
    for row_ind in range(start_row, end_row):
        chinese_data_in_row = sheet.row_values(row_ind)
        english_data_in_row = [translator.translate(str(chinese_text)) for chinese_text in chinese_data_in_row]
        english_data_chunk.append(english_data_in_row)
    data_english_version.extend(english_data_chunk)


cur_lang = 'zh'
trans_lang = 'en'

translator = t_trans(from_lang = cur_lang, to_lang =  trans_lang)
start_time = time.time()
data_english_version = []

# create a thread pool with the number of available CPU cores
num_threads = multiprocessing.cpu_count()
thread_pool = []

# divide the rows into chunks for each thread
chunk_size = int(sheet.nrows / num_threads)

# Create threads and assign work
for thread_index in range(num_threads):
    start_row = thread_index * chunk_size
    end_row = min((thread_index + 1) * chunk_size, sheet.nrows)  # Ensuring last chunk isn't out of bounds

    thread = threading.Thread(target=translate_chunk, args=(start_row, end_row))
    thread_pool.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in thread_pool:
    thread.join()

# Writing to output folder (no change needed here)
workbook = xlwt.Workbook()
worksheet = workbook.add_sheet("Translated Sheet 1.1")
for row_index, row_data in enumerate(data_english_version):
    for col_index, cell_value in enumerate(row_data):
        worksheet.write(row_index, col_index, cell_value)
workbook.save("./output/Translated_Exports_via_translate_THREAD.xls")

end_time = time.time()
trans_time = (end_time - start_time)
print("Total time taken by translate and saving xls file:", trans_time, "seconds")



Total time taken by translate and saving xls file: 1714.7480189800262 seconds


### Using `googletrans` module

We need to install googletrans version 4.0.0 rc1, normal version does not work for some system. [Read here](https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group).

Below code just is for checking if string is actually a string or not

#### NOTE:-
Using googletrans many a times leads to read operation time outs if too many operations are done continuously though they mention it allows unlimited calls.

In [ ]:
import re

def checkIfString(s):
    if type(s) != str:
        return False
    else:
        s = s.strip()
        reg = re.search(r"[-+]?\d+(\.\d+)?", s)
        if reg:
            num = reg.group()
            if(num == s):
                return False
            else:
                return True
        else:
            return True

In [ ]:
#trans_lang = 'en' # let googletrans auto detect input language

# Creating Translator object
translator = google_trans()
# print(type(translator.translate('请在外包装上注明,DUB 12817 +STDXB BY SEA, 巩固 与女士交谈 Atali 发货前', dest='en').text))
# providing an empty string or spaces causes error hence we cannot pass empty string
# print(translator.translate('   ', dest='en'))
# print(translator.translate('1234', dest='en').text)

start_time = time.time()

data_english_version = []
for row_ind in range(sheet.nrows):
    chinese_data_in_row = sheet.row_values(row_ind)
    english_data_in_row = []
    for chinese_text in chinese_data_in_row:
        try:

            if(str(chinese_text).strip()!='' and checkIfString(chinese_text)):
                translation = translator.translate(str(chinese_text)).text #default is english
            else:
                translation = chinese_text #keep it the same
            english_data_in_row.append(translation)
        except Exception as e:
            print(f"Error translating: {chinese_text}")
            print(f"Error details: {e}")
            english_data_in_row.append(chinese_text)  # Append original string on error
    data_english_version.append(english_data_in_row)


# Writing to output folder
workbook = xlwt.Workbook()
worksheet = workbook.add_sheet("Translated Sheet 2")
for row_index, row_data in enumerate(data_english_version):
    for col_index, cell_value in enumerate(row_data):
        worksheet.write(row_index, col_index, cell_value)
workbook.save("./output/Translated_Exports_via_googletrans.xls")

end_time = time.time()
googletrans_time = (end_time - start_time)
print("Total time taken by googletrans and saving xls file : ", googletrans_time," seconds")

Error translating: 等待买家确认收货
Error details: The read operation timed out
Error translating: 条
Error details: The read operation timed out
Error translating: M297
Error details: The read operation timed out
Error translating: 请在外包装上注明,DUB 12817 +STDXB BY AIR
Error details: The read operation timed out
Error translating: tb582836650
Error details: The read operation timed out
Total time taken by googletrans and saving xls file :  716.3989562988281  seconds
